# INFO 212: Data Science Programming 1
___

### Week 6: Data Loading, Storage, and File Formats
___

### Mon., May 11, 2020
---

**Question:**
- How to interact with external data sources using Python? 

**Objectives:**
- Read and write data from text formats
- Read and write data using different delimiters
- Read and write JSON data
- Read and write HTML and XML data
- Scrape data from the Web
- Read and write data in binary formats

## Course Project: Teams and Proposal 
Any questions about project teams and proposal?

# Loading Data
## Accessing data is a necessary first step for data analysis. We are going to be focused on data input and output using pandas, though there are numerous tools in other libraries to help with reading and writing data in various formats. Input and output typically falls into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
%matplotlib inline

# XML and HTML: Web Scraping
### Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.

### Pandas has a built-in function, read_html, which uses libraries like lxml and Beautiful Soup to automatically parse tables out of HTML files as DataFrame objects. To show how this works, I downloaded an HTML file (used in the pandas documentation) from the United States FDIC government agency showing bank failuresm (https://www.fdic.gov/bank/individual/failed/banklist.html). First, you must install some additional libraries used by read_html:

In [4]:
!pip install lxml beautifulsoup4 html5lib

Could not build wheels for lxml, since package 'wheel' is not installed.
Could not build wheels for beautifulsoup4, since package 'wheel' is not installed.
Could not build wheels for html5lib, since package 'wheel' is not installed.
Could not build wheels for soupsieve, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.
Could not build wheels for webencodings, since package 'wheel' is not installed.


In [5]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

ValueError: No tables found

In [6]:
tables.head()

NameError: name 'tables' is not defined

In [ ]:
type(tables)

In [ ]:
len(tables)

In [ ]:
failures = tables[0]
failures.head()

In [ ]:
failures.shape

In [ ]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [ ]:
type(close_timestamps[0])

In [ ]:
close_timestamps.dt.year.value_counts()

In [ ]:
close_timestamps.dt.year.value_counts().sort_index()

# JSON Data
## JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example.

In [ ]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

## How to parse a JSON structure into a Python object?

In [ ]:
import json
result = json.loads(obj)
type(result)

In [ ]:
result

## How to convert a Python object back to JSON?

In [ ]:
asjson = json.dumps(result)
asjson

In [ ]:
type(asjson)

## How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields.

In [ ]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

## The pandas `read_json` can automatically convert JSON datasets in specific arrangements into a Series or DataFrame.

In [ ]:
!cat examples/example.json

# Note on Possible Error Message for !cat
If you see an error message such as
```
'cat' is not recognized as an internal or external command,
operable program or batch file.
```
You need to restart your jupyter notebook using Git Bash which comes with Git for Windows. I asked you to install Git for Windows at the beginning of the quarter: https://git-scm.com/download/win

In [ ]:
data = pd.read_json('examples/example.json')
data

In [ ]:
print(data.to_json())
print(data.to_json(orient='records'))

# Binary Data Formats

## One of the easiest ways to store data (also known as serialization) efficiently in binary format is using Python’s built-in pickle serialization. pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [ ]:
frame = pd.read_csv('examples/ex1.csv')
frame

In [ ]:
frame.to_pickle('examples/frame_pickle')

In [ ]:
pd.read_pickle('examples/frame_pickle')

In [ ]:
!rm examples/frame_pickle

# Reading Microsoft Excel Files

## pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or pandas.read_excel function. Internally these tools use the add-on packages xlrd and openpyxl to read XLS and XLSX files, respectively.

## To use ExcelFile, create an instance by passing a path to an xls or xlsx file:

In [ ]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

## Data stored in a sheet can then be read into DataFrame with parse:

In [ ]:
pd.read_excel(xlsx, 'Sheet1')

## If you are reading multiple sheets in a file, then it is faster to create the ExcelFile, but you can also simply pass the filename to pandas.read_excel:

In [ ]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

## To write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using pandas objects’ to_excel method:

In [ ]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

## You can also pass a file path to to_excel and avoid the ExcelWriter:

In [ ]:
frame.to_excel('examples/ex2.xlsx')

In [ ]:
!rm examples/ex2.xlsx

# Interacting with Web APIs

## Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method recommended is the requests package.

## To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

In [ ]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

## The Response object’s json method will return a dictionary containing JSON parsed into native Python objects:

In [ ]:
data = resp.json()
data[0]['title']

## Each element in data is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass data directly to DataFrame and extract fields of interest:

In [ ]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

# Hierarchical Indexing

## Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form. Let’s start with a simple example; create a Series with a list of lists (or arrays) as the index.

```
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data
```

## What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The “gaps” in the index display mean “use the label directly above”:

```
data.index
```

## With a hierarchically indexed object, so-called partial indexing is possible, enabling you to concisely select subsets of the data:

```
data['b']
data['b':'c']
data.loc[['b', 'd']]
```

## Selection is even possible from an “inner” level:

```
data.loc[:, 2]
```

## Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. For example, you could rearrange the data into a DataFrame using its unstack method:

```
data.unstack()
```

## The inverse operation of unstack is stack:

```
data.unstack().stack()
```

## With a DataFrame, either axis can have a hierarchical index:

```
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame
```

## The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

```
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame
```

## With partial column indexing you can similarly select groups of columns:

```
frame['Ohio']
```

## A MultiIndex can be created by itself and then reused; the columns in the preceding DataFrame with level names could be created like this:

```
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])
```

# Reordering and Sorting Levels
## At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The swaplevel takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered):

```
frame.swaplevel('key1', 'key2')
```

## sort_index, on the other hand, sorts the data using only the values in a single level. When swapping levels, it’s not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level:

```
frame.sort_index(level=1)
frame.swaplevel(0, 1).sort_index(level=0)
```

# Summary Statistics by Level
## Many descriptive and summary statistics on DataFrame and Series have a level option in which you can specify the level you want to aggregate by on a particular axis. Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:

```
frame.sum(level='key2')
frame.sum(level='color', axis=1)
```

# Indexing with a DataFrame's columns
## It’s not unusual to want to use one or more columns from a DataFrame as the row index; alternatively, you may wish to move the row index into the DataFrame’s columns. Here’s an example DataFrame:


```
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame
````

## DataFrame’s set_index function will create a new DataFrame using one or more of its columns as the index:

```
frame2 = frame.set_index(['c', 'd'])
frame2
```

## By default the columns are removed from the DataFrame, though you can leave them in:

```
frame.set_index(['c', 'd'], drop=False)
```

## reset_index, on the other hand, does the opposite of set_index; the hierarchical index levels are moved into the columns:

```
frame2.reset_index()
```

# Combining and Merging Datasets

## Database-Style DataFrame Joins
## Merge or join operations combine datasets by linking rows using one or more keys. These operations are central to relational databases (e.g., SQL-based). The merge function in pandas is the main entry point for using these algorithms on your data. Let’s start with a simple example:

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

In [ ]:
df2

## This is an example of a many-to-one join; the data in df1 has multiple rows labeled a and b, whereas df2 has only one row for each value in the key column. Calling merge with these objects we obtain:

In [ ]:
pd.merge(df1, df2)

## Note that I didn’t specify whichcolumn to join on. If that information is not specified, merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [ ]:
pd.merge(df1, df2, on='key')

## If the column names are different in each object, you can specify them separately:

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

## You may notice that the 'c' and 'd' values and associated data are missing from the result. By default merge does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are 'left', 'right', and 'outer'. The outer join takes the union of the keys, combining the effect of applying both left and right joins:

In [ ]:
pd.merge(df1, df2, how='outer')

## Many-to-many merges have well-defined, though not necessarily intuitive, behavior. Here’s an example:

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1
df2
pd.merge(df1, df2, on='key', how='left')

## Many-to-many joins form the Cartesian product of the rows. Since there were three 'b' rows in the left DataFrame and two in the right one, there are six 'b' rows in the result. The join method only affects the distinct key values appearing in the result:

In [ ]:
pd.merge(df1, df2, how='inner')

## To merge with multiple keys, pass a list of column names:

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

## A last issue to consider in merge operations is the treatment of overlapping column names. While you can address the overlap manually (see the earlier section on renaming axis labels), merge has a suffixes option for specifying strings to append to overlapping names in the left and right DataFrame objects:

In [ ]:
pd.merge(left, right, on='key1')

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

## Merging on Index
## In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key:

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1

In [ ]:
right1

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True)

## Since the default merge method is to intersect the join keys, you can instead form the union of them with an outer join:

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

## With hierarchically indexed data, things are more complicated, as joining on index is implicitly a multiple-key merge:

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth

In [ ]:
righth

## In this case, you have to indicate multiple columns to merge on as a list (note the handling of duplicate index values with how='outer'):

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

## Using the indexes of both sides of the merge is also possible:

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2

In [ ]:
right2

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

## DataFrame has a convenient join instance for merging by index. It can also be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns. In the prior example, we could have written:

In [ ]:
left2.join(right2, how='outer')

## In part for legacy reasons (i.e., much earlier versions of pandas), DataFrame’s join method performs a left join on the join keys, exactly preserving the left frame’s row index. It also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame:

In [ ]:
left1.join(right1, on='key')

## Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as an alternative to using the more general concat function described in the next section:

In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

In [ ]:
left2.join([right2, another])

In [ ]:

left2.join([right2, another], how='outer')

## Concatenating Along an Axis
## Another kind of data combination operation is referred to interchangeably as concatenation,binding, or stacking. NumPy’s concatenate function can do this with NumPy arrays:

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr

In [ ]:
np.concatenate([arr, arr], axis=1)

## In the context of pandas objects such as Series and DataFrame, having labeled axesenable you to further generalize array concatenation. In particular, you have a number of additional things to think about:
* If the objects are indexed differently on the other axes, should we combine the
distinct elements in these axes or use only the shared values (the intersection)?
* Do the concatenated chunks of data need to be identifiable in the resulting
object?
* Does the “concatenation axis” contain data that needs to be preserved? In many
cases, the default integer labels in a DataFrame are best discarded during
concatenation.

## The concat function in pandas provides a consistent way to address each of theseconcerns. Here is a list of examples to illustrate how it works. Suppose we have three Series with no index overlap:

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

## Calling concat with these objects in a list glues together the values and indexes:

In [ ]:
pd.concat([s1, s2, s3])

## By default concat works along axis=0, producing another Series. If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):

In [ ]:
pd.concat([s1, s2, s3], axis=1)

## In this case there is no overlap on the other axis, which as you can see is the sorted union (the 'outer' join) of the indexes. You can instead intersect them by passing join='inner':

In [ ]:
s4 = pd.concat([s1, s3])
s4

In [ ]:
pd.concat([s1, s4], axis=1)

In [ ]:
pd.concat([s1, s4], axis=1, join='inner')

## In this last example, the 'f' and 'g' labels disappeared because of the join='inner' option.

## A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the keys argument:

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

In [ ]:
result.unstack()

## In the case of combining Series along axis=1, the keys become the DataFrame column headers:

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

## The same logic extends to DataFrame objects:

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1

In [ ]:
df2

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

In [ ]:
pd.concat([df1, df2], axis = 1)

## If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys option:

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

## There are additional arguments governing how the hierarchical index is created. For example, we can name the created axis levels with the names argument:

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

## A last consideration concerns DataFrames in which the row index does not contain any relevant data:

In [ ]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1

In [ ]:
df2

## In this case, you can pass ignore_index=True:

In [ ]:
pd.concat([df1, df2], ignore_index=True)

# GroupBy Mechanics

## Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly pivot tables for reporting or visualization purposes. pandas provides a flexible groupby interface, enabling you to slice, dice, and summarize datasets in a natural way.

![](https://i.imgur.com/ySkeo12.png)


## Each grouping key can take many forms, and the keys do not have to be all of the same type:
* A list or array of values that is the same length as the axis being grouped
* A value indicating a column name in a DataFrame
* A dict or Series giving a correspondence between the values on the axis being grouped and the group names
* A function to be invoked on the axis index or the individual labels in the index

## Note that the latter three methods are shortcuts for producing an array of values to be used to split up the object.

```
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df
```

## Suppose you wanted to compute the mean of the data1 column using the labels from key1. There are a number of ways to do this. One is to access data1 and call groupby with the column (a Series) at key1:

```
grouped = df['data1'].groupby(df['key1'])
grouped
```

## This grouped variable is now a GroupBy object. It has not actually computed anything yet except for some intermediate data about the group key df['key1']. The idea is that this object has all of the information needed to then apply some operation to each of the groups. For example, to compute group means we can call the GroupBy’s mean method:

```
grouped.mean()
```

## If instead we had passed multiple arrays as a list, we’d get something different:

```
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
```

## Here we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of the unique pairs of keys observed:

```
means.unstack().stack()
```

## In this example, the group keys are all Series, though they could be any arrays of the right length:

```
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()
```

## Frequently the grouping information is found in the same DataFrame as the data you want to work on. In that case, you can pass column names (whether those are strings, numbers, or other Python objects) as the group keys:

```
df.groupby('key1').mean()
```

```
df.groupby(['key1', 'key2']).mean()
```

## You may have noticed in the first case df.groupby('key1').mean() that there is no key2 column in the result. Because df['key2'] is not numeric data, it is said to be a nuisance column, which is therefore excluded from the result. By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset, as you’ll see soon.

## Regardless of the objective in using groupby, a generally useful GroupBy method is size, which returns a Series containing group sizes:

```
df.groupby(['key1', 'key2']).size()
```

## Iterating Over Groups
## The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. Consider the following:

```
for name, group in df.groupby('key1'):
    print(name)
    print(group)
```

## In the case of multiple keys, the first element in the tuple will be a tuple of key values:

```
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
```

## Of course, you can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dict of the data pieces as a one-liner:

```
pieces = dict(list(df.groupby('key1')))
pieces['b']
```

## By default groupby groups on axis=0, but you can group on any of the other axes. For example, we could group the columns of our example df here by dtype like so:

```
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)
```

## We can print out the groups like so:

```
for dtype, group in grouped:
    print(dtype)
    print(group)
```

## Selecting a Column or Subset of Columns
## Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:

```
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
## are syntactic suger for:

```
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```

## Especially for large datasets, it may be desirable to aggregate only a few columns. For example, in the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:

```
df.groupby(['key1', 'key2'])[['data2']].mean()
```

## The object returned by this indexing operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single column name is passed as a scalar:

```
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()
```

## Grouping with Dicts and Series
## Grouping information may exist in a form other than an array. Let’s consider another example DataFrame:

```
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people
```

## Now, suppose I have a group correspondence for the columns and want to sum together the columns by group:

```
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
```

## Now, you could construct an array from this dict to pass to groupby, but instead we can just pass the dict (I included the key 'f' to highlight that unused grouping keys are OK):

```
by_column = people.groupby(mapping, axis=1)
by_column.sum()
```

## The same functionality holds for Series, which can be viewed as a fixed-size mapping:

```
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()
```

## Grouping with Functions
## Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. Any function passed as a group key will be called once per index value, with the return values being used as the group names. More concretely, consider the example DataFrame from the previous section, which has people’s first names as index values. Suppose you wanted to group by the length of the names; while you could compute an array of string lengths, it’s simpler to just pass the len function:

```
people.groupby(len).sum()
```

## Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

```
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()
```

## Grouping by Index Levels
## A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index. Let’s look at an example:

```
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df
```

## To group by level, pass the level number or name using the level keyword:

```
hier_df.groupby(level='cty', axis=1).count()
```

# Reading and Writing Data in Text Format
## pandas features a number of functions for reading tabular data as a DataFrame object. The methods `read_csv` and `read_table` are likely the ones we’ll use the most.

In [ ]:
!cat examples/ex1.csv

In [ ]:
df = pd.read_csv('examples/ex1.csv', index_col=0)
df

## How to use `read_table()` to read a csv file?

In [ ]:
pd.read_table('examples/ex1.csv', sep=',')

## How to read a csv file without a header row?

In [ ]:
!cat examples/ex2.csv

## Let the Pandas to assign default header:

In [ ]:
pd.read_csv('examples/ex2.csv', header=None)

## Or specify your own header:

In [ ]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names)

## How to define a particular column as the index?

In [ ]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

## How to use multiple columns for index?

In [ ]:
df = pd.read_csv('examples/ex2.csv', names=names, index_col=['message', 'a'])

In [ ]:
df.index

In [ ]:
df

## How to read a text file with irregular delimiters?
- Answer: we can pass in a regular expression for the keyword 'sep'.

In [ ]:
list(open('examples/ex3.txt'))

In [ ]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

## The parser functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with skiprows:

In [ ]:
!cat examples/ex4.csv

In [ ]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

## How to handle missing data in the input file?

In [ ]:
!cat examples/ex5.csv

In [ ]:
result = pd.read_csv('examples/ex5.csv')
result

In [ ]:
pd.isnull(result)

## You can specify `na_values` for treating missing value:

In [ ]:
result = pd.read_csv('examples/ex5.csv', na_values=[1])
result

## Different NA sentinels can be specified for each column in a dict:

In [ ]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

## Reading Text Files in Pieces
## How to read a certain number of lines from a file?

In [ ]:
!wc  -l examples/ex6.csv

In [ ]:
pd.read_csv('examples/ex6.csv', nrows=10)

## Working with Delimited Formats
## It’s possible to load most forms of tabular data from disk using functions like pandas `read_table`. In some cases, however, some manual processing may be necessary. It’s not uncommon to receive a file with one or more malformed lines that trip up read_table. To illustrate the basic tools, consider a small CSV file:

In [ ]:
!cat examples/ex7.csv

In [ ]:
pd.read_csv("examples/ex7.csv")

## For any file with a single-character delimiter, you can use Python’s built-in csv module. To use it, pass any open file or file-like object to csv.reader:

In [ ]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [ ]:
for line in reader:
    print(line)

## From there, it’s up to you to do the wrangling necessary to put the data in the form that you need it.

In [ ]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [ ]:
header, values = lines[0], lines[1:]

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

## Writing Data to Text Format
## Data can also be exported to a delimited format.

In [ ]:
data = pd.read_csv('examples/ex5.csv')
data

In [ ]:
data.to_csv('examples/out.csv', sep='|', na_rep='NULL')
!cat examples/out.csv

## How to write csv file using different deliminter?

In [ ]:
import sys
data.to_csv(sys.stdout, sep='|', na_rep='NULL')

## How to write null values explicitly?

In [ ]:
data.to_csv(sys.stdout, na_rep='NULL')

## How to ignore index and header labels in the written file?

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

## How to write out a subset of the columns?

In [ ]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])